In [1]:
# Import necessary libraries
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import numpy as np
import os

In [2]:
# Define transformations for the training, validation, and test sets.
# Includes data augmentation for the training set, and resizing for the validation and test sets.
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Randomly resize and crop to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

val_test_transform = transforms.Compose([
    transforms.Resize(256),  # Resize the image so the shorter side is 256
    transforms.CenterCrop(224),  # Crop the image to 224x224 at the center
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize using ImageNet stats
])

In [3]:
# Load the dataset from the directories and apply the transformations.
# Datasets are assumed to be in 'data/train', 'data/val', and 'data/test'.
train_data = datasets.ImageFolder('data/train', transform=train_transform)
val_data = datasets.ImageFolder('data/val', transform=val_test_transform)
test_data = datasets.ImageFolder('data/test', transform=val_test_transform)

# Dataloaders for iterating over datasets. Batch size is 32.
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)  # Shuffle for training
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [4]:
# Load a pre-trained ResNet18 model and modify the final fully connected layer for binary classification.
# Use GPU if available, otherwise fall back to CPU.
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features  # Get the number of input features for the final layer
model.fc = nn.Linear(num_ftrs, 2)  # Modify the final layer for 2 classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move the model to the appropriate device

/media/vyom/DATA/vyom/School/Advanced Neural Networks Deep Learning For Spatial Analysis/pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/media/vyom/DATA/vyom/School/Advanced Neural Networks Deep Learning For Spatial Analysis/pytorch/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Define the loss function (cross-entropy for classification) and the Adam optimizer.
# The learning rate scheduler reduces the learning rate when validation loss plateaus.
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate of 0.001
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, verbose=True)  # Reduce LR on plateau

# Initialize TensorBoard writer for logging
writer = SummaryWriter('runs/leaf_classification')

# Early stopping criteria
early_stopping_patience = 4  # Stop training if no improvement for 4 consecutive epochs

/media/vyom/DATA/vyom/School/Advanced Neural Networks Deep Learning For Spatial Analysis/pytorch/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [6]:
# Training and validation loop
# Implements early stopping, learning rate scheduling, and TensorBoard logging.
num_epochs = 25
best_model_wts = model.state_dict()
best_acc = 0.0
early_stop_counter = 0

for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs-1}')
    print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # Set model to training mode
            dataloader = train_loader
        else:
            model.eval()  # Set model to evaluation mode
            dataloader = val_loader

        running_loss = 0.0
        running_corrects = 0

        # Iterate over the data
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # Backward and optimize only during training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # Track statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_acc = running_corrects.double() / len(dataloader.dataset)

        # Log the metrics to TensorBoard
        writer.add_scalar(f'{phase} Loss', epoch_loss, epoch)
        writer.add_scalar(f'{phase} Accuracy', epoch_acc, epoch)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Adjust learning rate only on validation phase
        if phase == 'val':
            scheduler.step(epoch_loss)

            # Save the best model based on validation accuracy
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
                early_stop_counter = 0  # Reset early stopping counter if accuracy improves
            else:
                early_stop_counter += 1

    # Early stopping check
    if early_stop_counter >= early_stopping_patience:
        print("Early stopping triggered")
        break

Epoch 0/24
----------
train Loss: 0.2292 Acc: 0.9175
val Loss: 0.7187 Acc: 0.8963
Epoch 1/24
----------
train Loss: 0.1228 Acc: 0.9580
val Loss: 0.3577 Acc: 0.9333
Epoch 2/24
----------
train Loss: 0.0954 Acc: 0.9664
val Loss: 0.0551 Acc: 0.9778
Epoch 3/24
----------
train Loss: 0.0868 Acc: 0.9748
val Loss: 0.5700 Acc: 0.7630
Epoch 4/24
----------
train Loss: 0.1132 Acc: 0.9497
val Loss: 0.0745 Acc: 0.9778
Epoch 5/24
----------
train Loss: 0.0830 Acc: 0.9776
val Loss: 0.0174 Acc: 0.9852
Epoch 6/24
----------
train Loss: 0.1013 Acc: 0.9706
val Loss: 0.0498 Acc: 0.9778
Epoch 7/24
----------
train Loss: 0.0686 Acc: 0.9790
val Loss: 0.1219 Acc: 0.9481
Epoch 8/24
----------
train Loss: 0.0975 Acc: 0.9692
val Loss: 0.0820 Acc: 0.9704
Epoch 9/24
----------
train Loss: 0.0957 Acc: 0.9622
val Loss: 0.0495 Acc: 0.9926
Epoch 10/24
----------
train Loss: 0.0926 Acc: 0.9804
val Loss: 0.0291 Acc: 0.9926
Epoch 11/24
----------
train Loss: 0.0698 Acc: 0.9790
val Loss: 0.0437 Acc: 0.9778
Epoch 12/24
--

In [7]:
# Load best model weights
model.load_state_dict(best_model_wts)

<All keys matched successfully>

In [8]:
# Evaluation on the test set
# Switch to evaluation mode and compute predictions
model.eval()

# Initialize lists to store labels and predictions
all_labels = []
all_preds = []

In [9]:
# Test loop
for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    
    all_labels.extend(labels.cpu().numpy())
    all_preds.extend(preds.cpu().numpy())

In [10]:
# Calculate metrics using scikit-learn
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='binary')
recall = recall_score(all_labels, all_preds, average='binary')
conf_matrix = confusion_matrix(all_labels, all_preds)

# Print the evaluation metrics for accuracy, precision, recall, and confusion matrix

In [11]:
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 1.0000


In [12]:
print(f'Precision: {precision:.4f}')

Precision: 1.0000


In [13]:
print(f'Recall: {recall:.4f}')

Recall: 1.0000


In [14]:
print(f'Confusion Matrix:\n{conf_matrix}')

Confusion Matrix:
[[3 0]
 [0 5]]


In [15]:
# Close the TensorBoard writer
writer.close()